# Advances Prompt Engineering Strategies

### Loading OpenAI API Key

In [2]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv(dotenv_path='../../.env')  # Specify the path to your .env file

# Access the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Check if the variable is loaded
if api_key or api_key == "":
    print("API key loaded successfully.")
else:
    print("Failed to load API key.")

API key loaded successfully.


## Initializing OpenAI API

In [3]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

## Auxiliary Functions

### Function to get the model response

In [4]:
class Message:
    def __init__(self, role, content):
        self.role = role
        self.content = content

def get_response(messages:list[Message], model="gpt-3.5-turbo", temperature=0.7):
    """
    Get a response from the OpenAI API.

    Parameters:
    - prompt (str): The input prompt for the model.
    - model (str): The model to use. Default is "gpt-3.5-turbo".
    - temperature (float): Sampling temperature. Default is 0.7.

    Returns:
    - str: The model's response.
    """
    model_response = client.chat.completions.create(
        model=model,
        messages=[message.__dict__ for message in messages],
        temperature=temperature
    )

    # Access the content of the first choice
    return model_response.choices[0].message.content

# Shots

## Zero-Shot, One-Shot, and Few-Shot Prompts

### Function to generate a prompt for extracting odd numbers

In [5]:
def get_extract_odd_numbers_prompt(input_data:set[int], examples:int=0):
    """
    Generate a prompt for the OpenAI API to extract odd numbers from a set of integers.
    :param input_data: A set of integers.
    :param examples: Number of examples to include in the prompt.
    :return: A string containing the prompt.
    """
    def get_question(question_data:set[int]):
        return f"Q: What are the odd numbers in the given set of integers {question_data}?"
    extract_odds_prompt = f"{get_question(input_data)}. A: "
    if examples > 0:
        shots = ""
        for i_exp in range(examples):
            example_data = set(range(i_exp * 2 + 1, i_exp * 2 + 3))
            odds = set(filter(lambda x: x % 2 != 0, example_data))
            shots += f"{get_question(example_data)}. A: {odds}\n"
        return shots + extract_odds_prompt
    return f"""{extract_odds_prompt}"""



In [6]:
# Example usage (only prompt)
data = {1, 2, 3, 4, 5, 6, 7, 8, 9}

for i in range(1, 4):
    prompt = get_extract_odd_numbers_prompt(data, examples=i)
    print(f"- Prompt with {i} example(s):\n{prompt}")
    print("-" * 50)

- Prompt with 1 example(s):
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {1, 2, 3, 4, 5, 6, 7, 8, 9}?. A: 
--------------------------------------------------
- Prompt with 2 example(s):
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {3, 4}?. A: {3}
Q: What are the odd numbers in the given set of integers {1, 2, 3, 4, 5, 6, 7, 8, 9}?. A: 
--------------------------------------------------
- Prompt with 3 example(s):
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {3, 4}?. A: {3}
Q: What are the odd numbers in the given set of integers {5, 6}?. A: {5}
Q: What are the odd numbers in the given set of integers {1, 2, 3, 4, 5, 6, 7, 8, 9}?. A: 
--------------------------------------------------


In [7]:
# Example usage (with response)
data = {3, 5, 11, 12, 16}
zero_shot_prompt = get_extract_odd_numbers_prompt(data)
response = get_response(messages=[Message(role="user", content=zero_shot_prompt)])
print(f"\nPrompt zero shot:\n{zero_shot_prompt}")
print(f"\nResponse zero shot:\n{response}")
print("-" * 50)

one_shot_prompt = get_extract_odd_numbers_prompt(data, examples=1)
response = get_response(messages=[Message(role="user", content=one_shot_prompt)])
print(f"\nPrompt one shot:\n{one_shot_prompt}")
print(f"\nResponse one shot:\n{response}")
print("-" * 50)

few_shot_prompt = get_extract_odd_numbers_prompt(data, examples=2)
response = get_response(messages=[Message(role="user", content=few_shot_prompt)])
print(f"\nPrompt few shot:\n{few_shot_prompt}")
print(f"\nResponse few shot:\n{response}")
print("-" * 50)


Prompt zero shot:
Q: What are the odd numbers in the given set of integers {16, 3, 5, 11, 12}?. A: 

Response zero shot:
3, 5, 11
--------------------------------------------------

Prompt one shot:
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {16, 3, 5, 11, 12}?. A: 

Response one shot:
{3, 5, 11}
--------------------------------------------------

Prompt few shot:
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {3, 4}?. A: {3}
Q: What are the odd numbers in the given set of integers {16, 3, 5, 11, 12}?. A: 

Response few shot:
{3, 5, 11}
--------------------------------------------------


### Sentiment analysis with few-shot prompting

#### Function to generate messages for sentiment analysis

In [8]:
def get_sentiment_analysis_messages(input_text:str, examples:int=0):
    """
    Generate messages for sentiment analysis.
    :param input_text: The input text for sentiment analysis.
    :param examples: Number of examples to include in the prompt.
    :return: A list of messages for the OpenAI API.
    """
    if examples > 6:
        raise ValueError("The maximum number of examples is 6.")

    messages = []
    if examples > 0:
        sentiment_examples = [
            {"sentence": "I love programming!", "sentiment": "Positive"},
            {"sentence": "I hate programming!", "sentiment": "Negative"},
            {"sentence": "This is a great day", "sentiment": "Positive"},
            {"sentence": "This is a bad day", "sentiment": "Negative"},
            {"sentence": "I am very happy.", "sentiment": "Positive"},
            {"sentence": "I am very sad.", "sentiment": "Negative"}
        ]
        for i_exp in range(examples):
            # Example data, question as user
            msg = Message(role="user", content=sentiment_examples[i_exp].get("sentence"))
            messages.append(msg)
            # Example data, answer as assistant
            msg = Message(role="assistant", content=sentiment_examples[i_exp].get("sentiment"))
            messages.append(msg)

    question_msg = Message(role="user", content=input_text)
    messages.append(question_msg)
    return messages

In [9]:
# Example usage (only prompt)
text = "I love programming!"
for i in range(1, 5):
    msgs = get_sentiment_analysis_messages(text, examples=i)
    print(f"\nMsgs with {i} example(s):\n{[msg.__dict__ for msg in msgs]}")
    print("-" * 50)


Msgs with 1 example(s):
[{'role': 'user', 'content': 'I love programming!'}, {'role': 'assistant', 'content': 'Positive'}, {'role': 'user', 'content': 'I love programming!'}]
--------------------------------------------------

Msgs with 2 example(s):
[{'role': 'user', 'content': 'I love programming!'}, {'role': 'assistant', 'content': 'Positive'}, {'role': 'user', 'content': 'I hate programming!'}, {'role': 'assistant', 'content': 'Negative'}, {'role': 'user', 'content': 'I love programming!'}]
--------------------------------------------------

Msgs with 3 example(s):
[{'role': 'user', 'content': 'I love programming!'}, {'role': 'assistant', 'content': 'Positive'}, {'role': 'user', 'content': 'I hate programming!'}, {'role': 'assistant', 'content': 'Negative'}, {'role': 'user', 'content': 'This is a great day'}, {'role': 'assistant', 'content': 'Positive'}, {'role': 'user', 'content': 'I love programming!'}]
--------------------------------------------------

Msgs with 4 example(s):


In [10]:
# Example usage (with response)
temp = 0.7
text = "The price of the product is really fair given its features"
zero_shot_msgs = get_sentiment_analysis_messages(text)
response = get_response(messages=zero_shot_msgs, temperature=temp)
print(f"\nResponse zero shot:\n{response}")
for zero_shot_msgs in zero_shot_msgs:
    print(f"- {zero_shot_msgs.__dict__}")
print("-" * 50)

one_shot_msgs = get_sentiment_analysis_messages(text, examples=1)
response = get_response(messages=one_shot_msgs, temperature=temp)
print(f"\nResponse one shot:\n{response}")
for one_shot_msgs in one_shot_msgs:
    print(f"- {one_shot_msgs.__dict__}")
print("-" * 50)

few_shot_msgs = get_sentiment_analysis_messages(text, examples=2)
response = get_response(messages=few_shot_msgs, temperature=temp)
print(f"\nResponse few shot:\n{response}")
for few_shot_msgs in few_shot_msgs:
    print(f"- {few_shot_msgs.__dict__}")
print("-" * 50)


Response zero shot:
 and quality. It offers great value for money.
- {'role': 'user', 'content': 'The price of the product is really fair given its features'}
--------------------------------------------------

Response one shot:
Positive
- {'role': 'user', 'content': 'I love programming!'}
- {'role': 'assistant', 'content': 'Positive'}
- {'role': 'user', 'content': 'The price of the product is really fair given its features'}
--------------------------------------------------

Response few shot:
Positive
- {'role': 'user', 'content': 'I love programming!'}
- {'role': 'assistant', 'content': 'Positive'}
- {'role': 'user', 'content': 'I hate programming!'}
- {'role': 'assistant', 'content': 'Negative'}
- {'role': 'user', 'content': 'The price of the product is really fair given its features'}
--------------------------------------------------


# Steps

## Single Step vs Multi-Step Prompts

### Single Step Prompt

In [11]:
single_prompt = """Make a plan for a beach vacation."""
response = get_response(messages=[Message(role="user", content=single_prompt)])
print(f"\nSingle-Step Prompt:\n{single_prompt}")
print(f"\nResponse:\n{response}")


Single-Step Prompt:
Make a plan for a beach vacation.

Response:
1. Choose a destination: Research different beach destinations and choose one that suits your preferences, whether it's a secluded tropical island, a bustling beach town, or a popular tourist destination.

2. Book accommodations: Look for beachfront hotels, resorts, or vacation rentals that fit your budget and desired amenities. Consider factors such as proximity to the beach, access to activities and attractions, and availability of dining options.

3. Plan activities: Research the activities available at your chosen destination, such as snorkeling, surfing, hiking, or simply relaxing on the beach. Make a list of must-try activities and book any necessary reservations in advance.

4. Pack essentials: Don't forget to pack essentials such as sunscreen, sunglasses, swimwear, beach towels, and flip-flops. Consider bringing a beach umbrella, portable cooler, and beach toys if needed.

5. Plan transportation: Arrange transpor

### Multi-Step Prompt

In [12]:
single_prompt = """Make a plan for a beach vacation as follows:
Step 1: List four potential locations.
Step 2: For each location, provide accommodation options.
Step 3: For each location, suggest activities.
Step 4: For each location, evaluate the pros and cons.
"""
response = get_response(messages=[Message(role="user", content=single_prompt)])
print(f"\nMulti-Step Prompt:\n{single_prompt}")
print(f"\nResponse:\n{response}")


Multi-Step Prompt:
Make a plan for a beach vacation as follows:
Step 1: List four potential locations.
Step 2: For each location, provide accommodation options.
Step 3: For each location, suggest activities.
Step 4: For each location, evaluate the pros and cons.


Response:
Step 1: 
1. Hawaii
2. Maldives
3. Bali, Indonesia
4. Cancun, Mexico

Step 2:
1. Hawaii - Accommodation options include luxury resorts, beachfront hotels, vacation rentals, and Airbnb.
2. Maldives - Accommodation options include overwater bungalows, luxury resorts, and private villas.
3. Bali, Indonesia - Accommodation options include beachfront hotels, boutique resorts, villas, and hostels.
4. Cancun, Mexico - Accommodation options include all-inclusive resorts, beachfront hotels, vacation rentals, and Airbnb.

Step 3:
1. Hawaii - Activities include snorkeling, surfing, hiking, exploring volcanoes, and relaxing on the beach.
2. Maldives - Activities include snorkeling, scuba diving, dolphin watching, water sports, 

In [13]:
code = '''
def calculate_rectangle_area(length, width):
    area = length * width
    return area
'''

prompt = f"""
Analyze the function delimited by triple backticks and determine if it is correct as follows:
Step 1: Correct syntax
Step 2: Receiving two inputs
Step 3: Returning one output
```{code}```
"""

response = get_response(messages=[Message(role="user", content=prompt)])
print(f"\nMulti-Step Prompt:\n{prompt}")
print(f"\nResponse:\n{response}")


Multi-Step Prompt:

Analyze the function delimited by triple backticks and determine if it is correct as follows:
Step 1: Correct syntax
Step 2: Receiving two inputs
Step 3: Returning one output
```
def calculate_rectangle_area(length, width):
    area = length * width
    return area
```


Response:
The function provided is correct based on the criteria given:
Step 1: The syntax is correct with the function definition, parameters, and return statement properly defined.
Step 2: The function is receiving two inputs: length and width.
Step 3: The function is returning one output, which is the calculated area of the rectangle.


In [14]:

def print_response(res, of:str):
    print(f"\nResponse of {of}:\n{res}")
    print(f"-" * 50)

# Build a chain-of-thought prompt to calculate a father's age in 10 years given a friend's current age.

# %%
prompt = (
    "Let’s think step by step. We know our friend is 20 years old, and their father is currently twice that age. "
    "First calculate the father’s current age, then determine how old he will be in 10 years by adding 10. "
    "Show all your reasoning, and then give the final answer."
)

response = get_response(messages=[Message(role="user", content=prompt)])
print_response(response, "Chain-of-Thought for Father's Age Calculation")


# Provide a worked example for summing even numbers in one set, then ask the model to apply the same reasoning to a new set.

# %%
example = """Q: Sum the even numbers in the following set: {9, 10, 13, 4, 2}.
A: Even numbers: 10, 4, 2. Adding them: 10 + 4 + 2 = 16"""
question = """Q: Sum the even numbers in the following set: {15, 13, 82, 7, 14}.
A:"""
prompt = example + "\n\n" + question
response = get_response(messages=[Message(role="user", content=prompt)])
print_response(response, "Worked Example for Summing Even Numbers")


# Use self-consistency by simulating three independent experts and choosing the majority answer for an inventory problem.

# %%
self_consistency_instruction = (
    "You will act as three independent reasoning experts. "
    "For each expert (Expert 1, Expert 2, Expert 3), generate a full chain-of-thought and a final answer to the problem below. "
    "Once all three answers are produced, compare them and choose the answer that at least two experts agree on, and present that as your final answer along with a brief note on the majority vote."
)
problem_to_solve = (
    "If you own a store that sells laptops and mobile phones. You start your day with "
    "50 devices in the store, out of which 60% are mobile phones. "
    "Throughout the day, three clients visited the store, each of them bought one mobile phone, "
    "and one of them bought additionally a laptop. "
    "Also, you added to your collection 10 laptops and 5 mobile phones. "
    "How many laptops and mobile phones do you have by the end of the day?"
)
prompt = self_consistency_instruction + "\n\nProblem:\n" + problem_to_solve
response = get_response(messages=[Message(role="user", content=prompt)])
print_response(response, "Chain of three independent reasoning experts")


# Refine a simple ask into a precise instruction that outputs a markdown table of the top ten pre-trained language models.

# %%
prompt = (
    "Create a markdown table with exactly three columns—“Model Name”, “Release Year”, and “Owning Company”—"
    "and ten rows, listing the top ten pre-trained language models. Include a header row and output only the table."
)
response = get_response(messages=[Message(role="user", content=prompt)])
print_response(response, "Chain of three independent reasoning experts")


# Adjust emotion-classification examples to include a “No explicit emotion” label and classify a neutral sentence.

# %%
prompt = """
Receiving a promotion at work made me feel on top of the world -> Happiness
The movie's ending left me with a heavy feeling in my chest -> Sadness
Walking alone in the dark alley sent shivers down my spine -> Fear
The clouds drifted lazily across the sky without stirring any feeling -> No explicit emotion

They sat and ate their meal -> No explicit emotion
"""
response = get_response(messages=[Message(role="user", content=prompt)])
print_response(response, "Chain of three independent reasoning experts")


# Summarize a market research report in up to five sentences, emphasizing AI and data privacy impacts on customers.

# %%
report = """
The market research report highlights the increasing influence of artificial intelligence (AI) and data privacy concerns on consumer behavior. AI technologies are being integrated into various sectors, enhancing customer experiences through personalized services and predictive analytics. However, the growing awareness of data privacy issues is prompting consumers to be more cautious about how their personal information is used, leading to a demand for greater transparency and control over data sharing. Companies that prioritize ethical AI practices and robust data protection measures are likely to gain a competitive edge in the market. Overall, the interplay between AI advancements and data privacy is reshaping consumer expectations and business strategies.
"""

prompt = f"""
Please summarize the following market research report in no more than five sentences, focusing specifically on how AI and data privacy are shaping the market and affecting customer behavior:

{report}
"""
response = get_response(messages=[Message(role="user", content=prompt)])
print_response(response, "Summarized Market Research Report")


# Generate at most five bullet points summarizing a smartphone’s key features for a review website.

# %%
product_description = """
Introducing the latest smartphone model, featuring a stunning 6.7-inch AMOLED display with a 120Hz refresh rate for smooth scrolling and vibrant colors. Powered by the latest octa-core processor, it ensures lightning-fast performance and seamless multitasking. Capture stunning photos with the triple-camera system, including a 108MP main sensor, ultra-wide lens, and telephoto capabilities. The device also boasts a long-lasting 5000mAh battery with fast charging support, ensuring you stay connected throughout the day. With 5G connectivity, advanced security features, and an elegant design, this smartphone is perfect for tech enthusiasts and everyday users alike.
"""

prompt = f"""
Please summarize the following smartphone product description in no more than five concise bullet points, focusing on the most important features:

{product_description}
"""
response = get_response(messages=[Message(role="user", content=prompt)])
print_response(response, "Smartphone Key Features Summary")


# Expand a concise smart home security camera description into one detailed paragraph wrapped in <DetailedDescription> tags and then list key points in a <Summary> block.

# %%
prompt = f"""
Expand the following concise product description of a smart home security camera into a detailed narrative in exactly one paragraph—highlighting its unique features, benefits, and potential applications—and wrap that paragraph in <DetailedDescription>…</DetailedDescription>.
Then provide a bullet-point summary of its key features, benefits, and applications wrapped in <Summary>…</Summary>:

{product_description}
"""
response = get_response(messages=[Message(role="user", content=prompt)])
print_response(response, "Smartphone Key Features Summary")



Response of Chain-of-Thought for Father's Age Calculation:
To find the father's current age, we first need to determine 2 times the age of our friend. Since our friend is 20 years old, we can calculate:

20 * 2 = 40

Therefore, the father is currently 40 years old.

To determine how old the father will be in 10 years, we simply need to add 10 to his current age:

40 + 10 = 50

Therefore, the father will be 50 years old in 10 years.
--------------------------------------------------

Response of Worked Example for Summing Even Numbers:
Even numbers: 82, 14. Adding them: 82 + 14 = 96.
--------------------------------------------------

Response of Chain of three independent reasoning experts:
Expert 1:
1. Start with 50 devices in the store, 60% of which are mobile phones: 50 * 0.6 = 30 mobile phones
2. Throughout the day, 3 clients bought 1 mobile phone each: 30 - 3 = 27 mobile phones left
3. One client bought a laptop, so subtract 1 laptop: 50 - 27 - 1 = 22 laptops left
4. Added 10 lap